In [13]:
import pandas as pd
import numpy as np
import os
import chardet
from google.colab import files
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns

# 마운트
from google.colab import drive
drive.mount('/content/drive')
print("마운트 완료")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
마운트 완료


In [14]:
# 인코딩 자동 감지 함수
def detect_encoding(file_path, n_lines=1000):
    with open(file_path, 'rb') as f:
        raw_data = b''.join([f.readline() for _ in range(n_lines)])
    return chardet.detect(raw_data)['encoding']

# 연도별 데이터 처리 함수
def process_year(year, base_path):
    print(f"\n{year}년 데이터 처리 시작")
    sales_path = os.path.join(base_path, f"상권_{year}.csv")
    temp_path = os.path.join(base_path, f"기온_{year}.csv")
    dust_path = os.path.join(base_path, f"미세먼지_{year}.csv")

    # 1. 상권 데이터
    sales_enc = detect_encoding(sales_path)
    df_sales = pd.read_csv(sales_path, encoding=sales_enc)
    df_sales['연도'] = df_sales['기준_년분기_코드'].astype(str).str[:4].astype(int)
    df_sales = df_sales[df_sales['연도'] == year]
    grouped = df_sales.groupby(['행정동_코드', '행정동_코드_명', '서비스_업종_코드_명'], as_index=False)['당월_매출_금액'].sum()
    grouped.columns = ['행정동코드', '행정동명', '업종', '총매출']
    grouped.insert(0, '연도', year)
    print(f"{year}년 상권 데이터: {grouped.shape}")
    print(grouped.head())

    # 2. 기온 데이터
    temp_enc = detect_encoding(temp_path)
    df_temp = pd.read_csv(temp_path, encoding=temp_enc)
    avg_temp = df_temp['평균기온(°C)'].dropna().mean()
    grouped['평균기온'] = round(avg_temp, 2)
    print(f"{year}년 평균 기온: {avg_temp:.2f}°C")

    # 3. 미세먼지 데이터
    dust_enc = detect_encoding(dust_path)
    df_dust = pd.read_csv(dust_path, encoding=dust_enc)
    pm_col = '미세먼지농도(㎍/㎥)' if '미세먼지농도(㎍/㎥)' in df_dust.columns else '미세먼지(㎍/㎥)'
    avg_dust = df_dust[pm_col].dropna().mean()
    grouped['평균미세먼지'] = round(avg_dust, 2)
    print(f"{year}년 평균 미세먼지: {avg_dust:.2f}㎍/㎥")

    return grouped

# 전체 연도 처리
def process_all_years(years, base_path):
    all_data = []
    for year in years:
        try:
            df = process_year(year, base_path)
            df.to_csv(os.path.join(base_path, f"통합_{year}.csv"), index=False, encoding='utf-8-sig')
            all_data.append(df)
            print(f"{year}년 처리 완료")
        except Exception as e:
            print(f"{year}년 처리 실패: {e}")

    if all_data:
        df_all = pd.concat(all_data, ignore_index=True)
        df_all.to_csv(os.path.join(base_path, "통합_2019_2024.csv"), index=False, encoding='utf-8-sig')
        print("전체 통합 파일 저장 완료: 통합_2019_2024.csv")
        files.download(os.path.join(base_path, "통합_2019_2024.csv"))
    return df_all

# 미래 값 예측 함수
def predict_future_value(df, col, year=2025):
    X = df[['연도']].values.reshape(-1, 1)
    y = df[col]
    model = LinearRegression().fit(X, y)
    pred = model.predict(np.array([[year]]))
    return round(pred[0], 2)

print("함수 정의 완료")

함수 정의 완료


In [15]:
base_path = '/content/drive/MyDrive/상권분석'
years = range(2019, 2025)
df_all = process_all_years(years, base_path)
print("\n통합 데이터프레임 정보:")
print(df_all.info())
print("\n통합 데이터프레임 샘플:")
print(df_all.head())


2019년 데이터 처리 시작
2019년 상권 데이터: (16970, 5)
     연도     행정동코드   행정동명     업종          총매출
0  2019  11110515  청운효자동    미용실    676679850
1  2019  11110515  청운효자동  분식전문점   2081525885
2  2019  11110515  청운효자동     서적  16233437173
3  2019  11110515  청운효자동   섬유제품    929224411
4  2019  11110515  청운효자동    세탁소    103455466
2019년 평균 기온: 13.60°C
2019년 평균 미세먼지: 41.76㎍/㎥
2019년 처리 완료

2020년 데이터 처리 시작
2020년 상권 데이터: (17167, 5)
     연도     행정동코드   행정동명     업종          총매출
0  2020  11110515  청운효자동   미곡판매     68137830
1  2020  11110515  청운효자동    미용실    584896798
2  2020  11110515  청운효자동  분식전문점   1599615632
3  2020  11110515  청운효자동     서적  16313220812
4  2020  11110515  청운효자동   섬유제품    614529599
2020년 평균 기온: 13.27°C
2020년 평균 미세먼지: 35.12㎍/㎥
2020년 처리 완료

2021년 데이터 처리 시작
2021년 상권 데이터: (18037, 5)
     연도     행정동코드   행정동명      업종         총매출
0  2021  11110515  청운효자동      가방    22673366
1  2021  11110515  청운효자동  가전제품수리    13837091
2  2021  11110515  청운효자동      문구    17747128
3  2021  11110515  청운효자동    미곡판매   12917

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


통합 데이터프레임 정보:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105390 entries, 0 to 105389
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   연도      105390 non-null  int64  
 1   행정동코드   105390 non-null  int64  
 2   행정동명    105390 non-null  object 
 3   업종      105390 non-null  object 
 4   총매출     105390 non-null  int64  
 5   평균기온    105390 non-null  float64
 6   평균미세먼지  105390 non-null  float64
dtypes: float64(2), int64(3), object(2)
memory usage: 5.6+ MB
None

통합 데이터프레임 샘플:
     연도     행정동코드   행정동명     업종          총매출  평균기온  평균미세먼지
0  2019  11110515  청운효자동    미용실    676679850  13.6   41.76
1  2019  11110515  청운효자동  분식전문점   2081525885  13.6   41.76
2  2019  11110515  청운효자동     서적  16233437173  13.6   41.76
3  2019  11110515  청운효자동   섬유제품    929224411  13.6   41.76
4  2019  11110515  청운효자동    세탁소    103455466  13.6   41.76


In [16]:
print("\n매출증감률 및 순위 계산 시작")
df_all = df_all.sort_values(by=['행정동코드', '업종', '연도'])
df_all['매출증감률'] = df_all.groupby(['행정동코드', '업종'])['총매출'].pct_change() * 100
df_all['순위'] = df_all.groupby(['연도', '행정동코드'])['총매출'].rank(method='min', ascending=False).astype(int)
df_all['업종코드'] = LabelEncoder().fit_transform(df_all['업종'])
df_all['행정동코드_str'] = LabelEncoder().fit_transform(df_all['행정동코드'].astype(str))
df_all['전년도_총매출'] = df_all.groupby(['행정동코드', '업종'])['총매출'].shift(1)

print("분석 데이터프레임 정보:")
print(df_all.info())
print("\n분석 데이터프레임 샘플:")
print(df_all.head())

df_all.to_csv(f"{base_path}/분석결과_2019_2024.csv", index=False, encoding='utf-8-sig')
print("분석 결과 저장 완료: 분석결과_2019_2024.csv")
files.download(f"{base_path}/분석결과_2019_2024.csv")


매출증감률 및 순위 계산 시작
분석 데이터프레임 정보:
<class 'pandas.core.frame.DataFrame'>
Index: 105390 entries, 34137 to 105389
Data columns (total 12 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   연도         105390 non-null  int64  
 1   행정동코드      105390 non-null  int64  
 2   행정동명       105390 non-null  object 
 3   업종         105390 non-null  object 
 4   총매출        105390 non-null  int64  
 5   평균기온       105390 non-null  float64
 6   평균미세먼지     105390 non-null  float64
 7   매출증감률      86055 non-null   float64
 8   순위         105390 non-null  int64  
 9   업종코드       105390 non-null  int64  
 10  행정동코드_str  105390 non-null  int64  
 11  전년도_총매출    86055 non-null   float64
dtypes: float64(4), int64(6), object(2)
memory usage: 10.5+ MB
None

분석 데이터프레임 샘플:
         연도     행정동코드   행정동명      업종       총매출   평균기온  평균미세먼지       매출증감률  순위  \
34137  2021  11110515  청운효자동      가방  22673366  13.75   37.90         NaN  35   
70094  2023  11110515  청운효자동      가방   

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [17]:
print("\n2025년 예측 시작")
df_mean = df_all.groupby('연도')[['평균기온', '평균미세먼지']].mean().reset_index()
예측기온 = predict_future_value(df_mean, '평균기온')
예측미세먼지 = predict_future_value(df_mean, '평균미세먼지')
print(f"2025년 예측 평균 기온: {예측기온}°C")
print(f"2025년 예측 평균 미세먼지: {예측미세먼지}㎍/㎥")

df_2024 = df_all[df_all['연도'] == 2024].copy()
df_2024['연도'] = 2025
df_2024['예측_총매출'] = np.nan
df_2024['평균기온'] = 예측기온
df_2024['평균미세먼지'] = 예측미세먼지
print("\n2025년 예측용 데이터프레임 샘플:")
print(df_2024.head())


2025년 예측 시작
2025년 예측 평균 기온: 14.67°C
2025년 예측 평균 미세먼지: 31.12㎍/㎥

2025년 예측용 데이터프레임 샘플:
         연도     행정동코드   행정동명      업종         총매출   평균기온  평균미세먼지      매출증감률  \
87848  2025  11110515  청운효자동  가전제품수리    33597594  14.67   31.12 -58.195754   
87849  2025  11110515  청운효자동      문구    12265218  14.67   31.12 -22.381544   
87850  2025  11110515  청운효자동    미곡판매    43687643  14.67   31.12   1.149426   
87851  2025  11110515  청운효자동     미용실  1084218623  14.67   31.12   4.274520   
87852  2025  11110515  청운효자동    반찬가게  1704000000  14.67   31.12   0.000000   

       순위  업종코드  행정동코드_str       전년도_총매출  예측_총매출  
87848  31     4          0  8.036886e+07     NaN  
87849  32    10          0  1.580194e+07     NaN  
87850  30    11          0  4.319119e+07     NaN  
87851  19    12          0  1.039773e+09     NaN  
87852  14    13          0  1.704000e+09     NaN  


In [18]:
# 학습 및 예측 데이터 통합
df_total = pd.concat([df_all, df_2024], ignore_index=True)
train = df_total[(df_total['연도'] < 2025) & (df_total['전년도_총매출'].notna())]
test = df_total[df_total['연도'] == 2025]

# 모델 훈련 및 예측
features = ['연도', '평균기온', '평균미세먼지', '전년도_총매출', '순위', '업종코드', '행정동코드_str']
X_train = train[features]
y_train = train['총매출']
X_test = test[features]

model = RandomForestRegressor(n_estimators=200, random_state=42)
model.fit(X_train, y_train)
test.loc[:, '예측_총매출'] = model.predict(X_test)

# 결과 정리
result = test[['연도', '행정동코드', '행정동명', '업종', '예측_총매출', '평균기온', '평균미세먼지', '전년도_총매출']].copy()
result['매출증감률_예측'] = ((result['예측_총매출'] - result['전년도_총매출']) / result['전년도_총매출']) * 100
result['순위'] = result.groupby('행정동코드')['예측_총매출'].rank(method='min', ascending=False).astype(int)
result = result.sort_values(['행정동코드', '순위'])
print("\n2025년 예측 결과 샘플:")
print(result.head())

# 최종 결과 저장 및 다운로드
result.to_csv(f"{base_path}/예측결과_2025.csv", index=False, encoding='utf-8-sig')
print("예측 결과 저장 완료: 예측결과_2025.csv")
files.download(f"{base_path}/예측결과_2025.csv")


2025년 예측 결과 샘플:
          연도     행정동코드   행정동명     업종        예측_총매출   평균기온  평균미세먼지  \
105417  2025  11110515  청운효자동  커피-음료  2.191746e+10  14.67   31.12   
105396  2025  11110515  청운효자동     서적  1.700435e+10  14.67   31.12   
105420  2025  11110515  청운효자동  한식음식점  1.386325e+10  14.67   31.12   
105404  2025  11110515  청운효자동  양식음식점  1.153230e+10  14.67   31.12   
105400  2025  11110515  청운효자동   슈퍼마켓  8.003662e+09  14.67   31.12   

             전년도_총매출   매출증감률_예측  순위  
105417  1.939355e+10  13.014140   1  
105396  1.705461e+10  -0.294743   2  
105420  1.379444e+10   0.498870   3  
105404  1.030842e+10  11.872664   4  
105400  8.347530e+09  -4.119404   5  
예측 결과 저장 완료: 예측결과_2025.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>